In [1]:
%load_ext autoreload
%autoreload 2
from autoseg.config import read_config
from autoseg.models import Model
import torch
import numpy as np
import matplotlib.pyplot as plt

/home/anton/miniforge3/envs/segmentation/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
config = read_config("autoseg/user_configs/anton/baselines/unet_convnext")

In [3]:
model = Model(config)

[[1, 8, 8], [1, 4, 4], [1, 2, 2]]


In [4]:
model.to("cuda:1")

Model(
  (model): ConfigurableUNet(
    (unet): UNet(
      (l_conv): ModuleList(
        (0): ConvNeXtPass(
          (passes): ModuleList(
            (0): ConvNeXtPassInner(
              (conv1): Conv3d(1, 1, kernel_size=(3, 3, 3), stride=(1, 1, 1))
              (norm): LayerNorm((1,), eps=1e-05, elementwise_affine=True)
              (conv2): Linear(in_features=1, out_features=24, bias=True)
              (act): GELU(approximate='none')
              (conv3): Linear(in_features=24, out_features=12, bias=True)
            )
            (1-3): 3 x ConvNeXtPassInner(
              (conv1): Conv3d(12, 12, kernel_size=(3, 3, 3), stride=(1, 1, 1))
              (norm): LayerNorm((12,), eps=1e-05, elementwise_affine=True)
              (conv2): Linear(in_features=12, out_features=24, bias=True)
              (act): GELU(approximate='none')
              (conv3): Linear(in_features=24, out_features=12, bias=True)
            )
          )
        )
        (1): ConvNeXtPass(
          (p

In [5]:
print(f"Num params: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

Num params: 14,534,817


In [6]:
model.load()

/home/anton/github/autoseg/src/autoseg/artifacts/UNet_convnext_3x3x3/checkpoints/best/ckpt.pt


In [7]:
from autoseg.datasets import GunpowderZarrDataset
from torch.utils.data import DataLoader
from autoseg.datasets.utils import multisample_collate as collate

In [8]:
dataset = GunpowderZarrDataset(
  config=config["pipeline"],
  input_image_shape=config["model"]["input_image_shape"],
  output_image_shape=config["model"]["output_image_shape"],
)

/home/anton/.cache/autoseg/datasets/SynapseWeb/kh2015/data/apical.zarr.zip labels/s1


/home/anton/.cache/autoseg/datasets/SynapseWeb/kh2015/data/apical.zarr.zip labels_mask/s1
/home/anton/.cache/autoseg/datasets/SynapseWeb/kh2015/data/apical.zarr.zip raw/s1
/home/anton/.cache/autoseg/datasets/SynapseWeb/kh2015/data/oblique.zarr.zip labels/s1
/home/anton/.cache/autoseg/datasets/SynapseWeb/kh2015/data/oblique.zarr.zip labels_mask/s1
/home/anton/.cache/autoseg/datasets/SynapseWeb/kh2015/data/oblique.zarr.zip raw/s1
/home/anton/.cache/autoseg/datasets/SynapseWeb/kh2015/data/spine.zarr.zip labels/s1
/home/anton/.cache/autoseg/datasets/SynapseWeb/kh2015/data/spine.zarr.zip labels_mask/s1
/home/anton/.cache/autoseg/datasets/SynapseWeb/kh2015/data/spine.zarr.zip raw/s1


In [9]:
dataloader = DataLoader(
    dataset=dataset,
    collate_fn=collate,
    batch_size=config["training"]["train_dataloader"]["batch_size"],
    pin_memory=False,
)

In [10]:
dataloader_it = iter(dataloader)

In [11]:
sample_image = next(dataloader_it)

In [12]:
sample_image[0].shape

(1, 1, 76, 304, 304)

In [13]:
raw_img = sample_image[0][0][0][24]

In [14]:
d3_raw_img = np.sum(sample_image[0][0][0][16:32], axis=0) / 16
d3_raw_img.shape
plt.imshow(d3_raw_img, cmap="gray")

NameError: name 'plt' is not defined

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(raw_img, cmap="gray")

In [ ]:
raw = torch.tensor(sample_image[0])
raw = raw.to("cuda")
raw.shape

In [ ]:
208*208*48/(16*16*16)

In [ ]:
model = model.to("cuda")

In [ ]:
layer = model.model.unet.transformer.layer[0]

In [ ]:
layer

In [ ]:
patch_embeddings = model.model.unet.transformer.embeddings.patch_embeddings(raw)

In [ ]:
patch_embeddings.flatten(2).shape

In [ ]:
patch_embeddings.shape

In [ ]:
def to_np(t):
  return t.detach().cpu().numpy()

In [ ]:
patch_embeddings = to_np(patch_embeddings)

In [ ]:
13*16

In [ ]:
# batch, emb_channel, z
plt.imshow(patch_embeddings[0,39,1])

In [ ]:
def patch_to_original_size(patch):
  return patch.repeat(16,axis=0).repeat(16,axis=1)

In [ ]:
from itertools import cycle

In [ ]:
values = cycle([0,1])
overlay = np.array([[next(values) for i in range(13)] for j in range(13)])

In [ ]:
plt.imshow(overlay)

In [ ]:
#plt.imshow(patch_to_original_size(2*patch_embeddings[0,39,1]) + raw_img,cmap="gray")
plt.imshow(patch_to_original_size(overlay) + 12*d3_raw_img,cmap="gray", vmin=d3_raw_img.min()*12, vmax=d3_raw_img.max()*12+1)
#plt.imshow(patch_to_original_size(overlay) + 4*raw_img,cmap="gray")

In [ ]:
plt.imshow(d3_raw_img[0*16:2*16,10*16:12*16], cmap="gray", vmin=d3_raw_img.min(), vmax=d3_raw_img.max())

In [ ]:
#num_heads * emb_dim
12*(768/12)
# (507, 768) * (768)

In [ ]:
embeddings = model.model.unet.transformer.embeddings(raw)

In [ ]:
embeddings.shape

In [ ]:
first_layer_out = layer.attn(layer.attention_norm(embeddings), return_raw_scores=True)


In [ ]:
attn_scores = first_layer_out[2]

In [ ]:
attn_scores = to_np(attn_scores)

In [ ]:
attn_scores.shape

In [ ]:
print(attn_scores.min(), attn_scores.max(), attn_scores.std())

In [ ]:
first_map_attn = attn_scores[0,s:s+c,0]

In [ ]:
torch.unflatten(torch.tensor(first_map_attn),dim=1,sizes=(3,13,13)).numpy().shape

In [ ]:
def attn_for_patch_i(attn,x,y,z):
  i = z*13*13+y*13+x
  p_attn = attn_scores[0,s:s+c,i]
  return np.transpose(torch.unflatten(torch.tensor(p_attn),dim=1,sizes=(3,13,13)).numpy(), (0, 2, 3, 1))

In [ ]:
attn_for_patch_i(attn_scores, 52).shape

In [ ]:
import numpy as np

In [ ]:
z=2
x=6
y=6
#print("Mean attention", attn_for_patch_i(attn_scores, x,y,z)[0,:,:,1].mean())
plt.imshow(attn_for_patch_i(attn_scores, x,y,z)[0,:,:,1], cmap="hot", vmin=-2,vmax=2)

In [ ]:
xs = 4
xe = 8 
ys = 8 
ye = 12
tot_attn = None
ct = 0
for x in range(xs,xe+1):
  for y in range(ys,ye+1):
    ct += 1
    if tot_attn is None:
      tot_attn = attn_for_patch_i(attn_scores, x,y,1)
    else:
      tot_attn += attn_for_patch_i(attn_scores, x,y,1)
tot_attn /= ct

In [ ]:
plt.imshow(d3_raw_img[ys*16:ye*16,xs*16:xe*16], cmap="gray", vmin=d3_raw_img.min(), vmax=d3_raw_img.max())

In [ ]:
# tot std: 0.08
# std only white patch: 0.28
# std white/black patch 0.13
print(tot_attn.min(), tot_attn.max(), tot_attn.std())

In [ ]:
plt.imshow(tot_attn[0,:,:,1], cmap="hot", vmin=-2,vmax=2)